**CheXpert-derived labels** were used for the following reasons:
  - Wider clinical coverage (more findings per study captured).
  - Multi-label ready (each study has 14 possible findings).
  - Includes uncertain cases (better mimics clinical reality).
  - CheXpert prioritization logic (positive > uncertain > negative) is more consistent with human diagnostic decision-making.
  - Standardized mention extraction patterns (including synonyms and abbreviations).

The following are the labels extracted using CheXpert:
- No Finding
- Enlarged Cardiomediastinum
- Cardiomegaly
- Lung Opacity                                            
- Lung Lesion
- Edema
- Consolidation
- Pneumonia
- Atelectasis
- Pneumothorax
- Pleural Effusion
- Pleural Other
- Fracture
- Support Devices


| Label Value | Meaning       | Interpretation                                    |
| ----------- | ------------- | ------------------------------------------------- |
| `1.0`       | Positive      | Condition is clearly **present**                 |
| `0.0`       | Negative      | Condition is clearly **absent**                   |
| `-1.0`      | Uncertain     | Radiologist **suspects** condition but isn't sure |
| `null`      | Not mentioned | No statement about the condition                  |


There are over 14 view positions for the chest X-rays in the dataset.

For chest X-rays, **PA (Posteroanterior)** and **AP (Anteroposterior)** are the most common and diagnostically rich frontal views:

- **PA** is the standard in ambulatory settings:
  - The patient stands facing the detector.
  - Results in a clearer and less magnified heart silhouette.

- **AP** is often used for bedridden or ICU patients:
  - Taken with the detector behind the patient.
  - More prone to artifacts, but still interpretable.

These views are preferred because:

- They provide a **full frontal projection** of the chest.
- They are **easier to learn from** due to greater dataset availability.
- They offer **more consistent anatomical display** across patients.
- They are **commonly labeled** as part of “normal” or “abnormal” classes in training datasets.

By **restricting training to PA/AP views**, we:
- **Reduce variability** caused by pose or projection differences.
- Help the model **focus on pathology**, rather than view-dependent artifacts.


In [1]:
import pandas as pd
from google.cloud import storage
import os
import shutil
import gcsfs
import io
import base64
# For local environment, use Application Default Credentials
# Make sure you've run: gcloud auth application-default login
# or set GOOGLE_APPLICATION_CREDENTIALS environment variable

In [15]:
bucket_name = "filtered_cxr"
file_path = "cxr"
gcs_path = f"gs://{bucket_name}/{file_path}"

df = pd.read_csv(gcs_path)
df.head()

,subject_id,study_id,dicom_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged_Cardiomediastinum,Fracture,Lung_Lesion,...,Pleural_Effusion,Pleural_Other,Pneumonia,Pneumothorax,Support_Devices,split,ViewPosition,Rows,Columns,study_datetime
0,14887088,54257662,1bc85033-355accce-e8d0ed50-78188cd3-dac92e86,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,train,AP,2539,776,2135-01-27 04:49:44.968
1,18650767,54780106,7920918a-6b7415f5-f5b6f7d5-815bb396-c6702eb3,NaN,-1.0,NaN,-1.0,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,train,PA,1504,1188,2131-09-14 22:14:49.221
2,11548266,59905684,95bf2c9c-fd2e6da5-fe82e5db-032420dd-d4daa45c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,train,PA,1713,1309,2129-04-20 13:37:47.133
3,13191147,55483625,b9cf3249-939bd85e-aaf9795b-d2c0a201-3c0d322c,NaN,NaN,NaN,0.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,train,PA,2000,1356,2184-12-08 12:03:51.120
4,13880916,53360233,729183be-b03670e1-35db29c1-356f54ed-94481994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,train,PA,1986,1380,2170-04-27 12:42:31.124


### Sampling images per lable

In [16]:
label_cols = [
    'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema',
    'Enlarged_Cardiomediastinum', 'Fracture', 'Lung_Lesion',
    'Lung_Opacity', 'No_Finding', 'Pleural_Effusion',
    'Pleural_Other', 'Pneumonia', 'Pneumothorax', 'Support_Devices'
]

# Filter to rows with at least one positive label
df_labels = df.copy()
df_labels = df_labels[df_labels[label_cols].notna().any(axis=1)]

# Container for sampled entries
sampled_rows = []

# Keep track of already used dicom_ids
used_dicom_ids = set()

# Sampling 5000 per label
for label in label_cols:
    label_df = df_labels[df_labels[label] == 1.0]
    # Avoid reusing the same dicom_id unless unavoidable
    label_df = label_df[~label_df['dicom_id'].isin(used_dicom_ids)]
    # Take up to 5000
    sample = label_df.sample(n=min(5000, len(label_df)), random_state=33)
    # Track used dicoms
    used_dicom_ids.update(sample['dicom_id'])
    sampled_rows.append(sample)

# Combine all sampled subsets
sampled_df = pd.concat(sampled_rows, ignore_index=True)

def label_to_text(label, condition):
    if pd.isna(label):
        return None
    elif label == 1.0:
        return f"Evidence of {condition.lower()} is present."
    elif label == 0.0:
        return f"No evidence of {condition.lower()}."
    elif label == -1.0:
        return f"There is suspicion of {condition.lower()}."
    else:
        return None

def create_report(row):
    sentences = []
    for condition in label_cols:
        sentence = label_to_text(row[condition], condition)
        if sentence:
            sentences.append(sentence)
    return " ".join(sentences)

# Add new column to the DataFrame
sampled_df["mini_report"] = sampled_df.apply(create_report, axis=1)

# Save final result
sampled_df.to_csv('sampled_5000_per_label.csv', index=False)

In [17]:
sampled_df.shape

(65726, 23)

In [18]:
sampled_df.head()

,subject_id,study_id,dicom_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged_Cardiomediastinum,Fracture,Lung_Lesion,...,Pleural_Other,Pneumonia,Pneumothorax,Support_Devices,split,ViewPosition,Rows,Columns,study_datetime,mini_report
0,18310719,57997889,84eb7e34-12b4cdcf-04196ef9-a5d6e59d-3d17fafc,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,train,AP,2544,3056,2129-05-30 17:47:45.468,Evidence of atelectasis is present.
1,14004638,54797919,110e4fe4-2b4d6b84-14687e5f-877c71ea-d469285a,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,train,AP,3050,2539,2156-01-24 08:14:06.953,Evidence of atelectasis is present.
2,13233757,56501945,43eaf4e8-7069e8c1-472726df-b9acc888-c8069f09,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,1.0,train,AP,2539,3050,2156-08-30 05:51:23.468,Evidence of atelectasis is present. Evidence o...
3,16444272,54476134,32e6ec27-1022a80a-37c2af58-4415c8de-d3da40ef,1.0,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,train,PA,3056,2544,2146-02-09 23:01:11.437,Evidence of atelectasis is present. Evidence o...
4,17372922,51457365,796cef17-0b455d0e-56288df5-80097fde-f090b2ba,1.0,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,train,AP,2544,3056,2156-10-05 20:30:24.765,Evidence of atelectasis is present. No evidenc...


In [21]:
sampled_df['image_data'] = None

In [22]:
billing_project = 'ablation-study'
fs = gcsfs.GCSFileSystem(project=billing_project, requester_pays=True)

In [ ]:
for idx, row in sampled_df.iterrows():
    sub = str(row['subject_id'])
    st  = str(row['study_id'])
    dic = str(row['dicom_id'])
    prefix = f"p{sub[:2]}"
    image_path = f'mimic-cxr-jpg-2.1.0.physionet.org/files/{prefix}/p{sub}/s{st}/{dic}.jpg'
    try:
        with fs.open(image_path, 'rb') as img_file:
            img_bytes = img_file.read()
            b64 = base64.b64encode(img_bytes).decode('utf-8')
            df.at[idx, 'image_data'] = b64
            print(f"✅ Embedded image for {dic}")
    except Exception as e:
        print(f"❌ Failed {dic}: {e}")
        df.at[idx, 'image_data'] = ""

# Save the new CSV to local Downloads directory
out_csv = "/Users/sv.xxt/Downloads/filtered_cohort_with_images.csv"
sampled_df.to_csv(out_csv, index=False)

print(f"✅ CSV with embedded images saved locally at: {out_csv}")

✅ Embedded image for 84eb7e34-12b4cdcf-04196ef9-a5d6e59d-3d17fafc
